In [1]:
import boto3
import json
from sagemaker import get_execution_role
from time import strftime
import calendar

In [2]:
%store -r
print(f"default_bucket : {default_bucket}")

default_bucket : sagemaker-us-west-2-322537213286


In [3]:
!aws s3 sync ./Informer2020/ s3://$default_bucket/Informer2020 --quiet

In [4]:
iam_client = boto3.client('iam')

In [5]:
role=get_execution_role()

In [6]:
base_role_name=role.split('/')[-1]

In [7]:
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'
)
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AWSLambda_FullAccess'
)
# iam_client.attach_role_policy(
#     RoleName=base_role_name,
#     PolicyArn='arn:aws:iam::aws:policy/service-role/AWSQuickSightListIAM'
# )
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::322537213286:policy/service-role/AWSQuickSightS3Policy'
)


{'ResponseMetadata': {'RequestId': 'eecdc0c2-71c2-4f46-9e67-b2fdca9b50a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eecdc0c2-71c2-4f46-9e67-b2fdca9b50a0',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Sun, 03 Oct 2021 08:52:46 GMT'},
  'RetryAttempts': 0}}

In [8]:
sm_client = boto3.client("sagemaker")

In [9]:
project_name='mlops-training'

In [10]:
project_response=sm_client.describe_project(ProjectName=project_name)

In [11]:
model_package_group_name = project_response['ProjectName']+"-"+project_response['ProjectId']

## 1. Create Amazon EventBridge Rule

In [12]:
event_client = boto3.client('events')

In [13]:
eventpattern = json.dumps(
    {
      "source": ["aws.sagemaker"],
      "detail-type": ["SageMaker Model Package State Change"],
      "detail": {
        "ModelPackageGroupName": [f"{model_package_group_name}"],
        "ModelApprovalStatus": ["Approved"]
      }
    }
)

In [14]:
rule_name = 'informer_model_package_state'
event_rule = event_client.put_rule(
    Name=rule_name,
    EventPattern=eventpattern,
    State='ENABLED',
    Description='This is after the approval update for the Informer model',
)

## 2. Create Lambda function

In [15]:
code_location = f"s3://{default_bucket}/sagemaker_lambda/"
zip_filename = 'lambda.zip'

In [16]:
%%bash
cd ./Informer2020/sagemaker_lambda/
zip lambda.zip create_model.py

updating: create_model.py (deflated 64%)


In [17]:
!aws s3 cp ./Informer2020/sagemaker_lambda/$zip_filename $code_location

upload: Informer2020/sagemaker_lambda/lambda.zip to s3://sagemaker-us-west-2-322537213286/sagemaker_lambda/lambda.zip


In [18]:
lambda_client = boto3.client('lambda')

In [19]:
lambda_trust_policy=json.dumps({
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
})

In [20]:
role_name='lambda-assume-role'
try:
    iam_client.detach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam_client.detach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )
    iam_client.delete_role(RoleName=role_name)
except:
    pass
finally:
    lambda_role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=lambda_trust_policy
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )

KMS 에러가 발생한 경우 lambda_function을 다시 생성합니다.

In [21]:
lambda_name='informer-model-creation-lambda-' + strftime("%m%d-%H%M%s")
try:
    lambda_client.delete_function(FunctionName=lambda_name)
except:
    pass
finally:
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_name,
        Runtime='python3.9',
        Role=lambda_role['Role']['Arn'],
        Handler='create_model.lambda_handler',
        Code={
            'S3Bucket': f'{default_bucket}',
            'S3Key': f'sagemaker_lambda/{zip_filename}'
        },
        Description='Create the latest version-based Informer model',
        Timeout=600,
        MemorySize=256,
        Environment={
          'Variables': {
              "role" : role,
              "default_bucket" : default_bucket
          }
      }
    )

In [22]:
lambda_permission_response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='InvokeLambdaFunction',
    Action='lambda:InvokeFunction',
    Principal="events.amazonaws.com",
    SourceArn=event_rule['RuleArn'],
)

## 3. Target 셋팅

In [23]:
event_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            'Id': 'Target0',
            'Arn': lambda_response['FunctionArn']
        }
    ]
)

{'FailedEntryCount': 0,
 'FailedEntries': [],
 'ResponseMetadata': {'RequestId': 'cace0138-2ab9-4033-94ec-8212b2e35381',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cace0138-2ab9-4033-94ec-8212b2e35381',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '41',
   'date': 'Sun, 03 Oct 2021 08:53:00 GMT'},
  'RetryAttempts': 0}}

### Studio 에서 실행

In [24]:
# !aws s3 sync s3://$default_bucket/Informer2020 ./Informer2020

## QuickSight

quicksight에서 사용할 manifest_file 생성합니다.

In [25]:
manifest_file = {
                "fileLocations": [
                    {
                        "URIPrefixes": [
                            f"s3://{default_bucket}/prediciton_result/"
                        ]
                    }
                ],
                "globalUploadSettings": {
                    "format": "CSV",
                    "delimiter": ",",
                    "textqualifier": "\"",
                    "containsHeader": "true"
                }
            }

In [ ]:
with open("./quicksight/manifest_file.json", 'w', encoding="utf-8") as f:
    json.dump(manifest_file, f, indent="\t")

In [ ]:
!aws s3 cp ./quicksight/manifest_file.json s3://$default_bucket/quick_sight/

### refresh-quicksight spice dataset

In [ ]:
qs_client = boto3.client("quicksight")

In [ ]:
sts_client = boto3.client("sts")

In [ ]:
account_id = sts_client.get_caller_identity()['Account']

In [ ]:
datasets_ids = ['f48083af-f596-4101-adfe-dfc07957f7ff']

In [ ]:
import uuid

In [ ]:
res = qs_client.list_data_sets(AwsAccountId=account_id)

# filter out your datasets using a prefix. All my datasets have chicago_crimes as their prefix
datasets_ids = [summary["DataSetId"] for summary in res["DataSetSummaries"]]
ingestion_ids = []

for dataset_id in datasets_ids:
    try:
        ingestion_id = str(calendar.timegm(time.gmtime()))
#         ingestion_id = str(uuid.uuid4())
        qs_client.create_ingestion(DataSetId=dataset_id, IngestionId=ingestion_id,
                                             AwsAccountId=account_id)
        ingestion_ids.append(ingestion_id)
    except Exception as e:
        print(e)
        pass

for ingestion_id, dataset_id in zip(ingestion_ids, datasets_ids):
    while True:
        response = qs_client.describe_ingestion(DataSetId=dataset_id,
                                             IngestionId=ingestion_id,
                                             AwsAccountId=account_id)
        if response['Ingestion']['IngestionStatus'] in ('INITIALIZED', 'QUEUED', 'RUNNING'):
            time.sleep(5)     #change sleep time according to your dataset size
        elif response['Ingestion']['IngestionStatus'] == 'COMPLETED':
            print("refresh completed. RowsIngested {0}, RowsDropped {1}, IngestionTimeInSeconds {2}, IngestionSizeInBytes {3}".format(
                response['Ingestion']['RowInfo']['RowsIngested'],
                response['Ingestion']['RowInfo']['RowsDropped'],
                response['Ingestion']['IngestionTimeInSeconds'],
                response['Ingestion']['IngestionSizeInBytes']))
            break
        else:
            print("refresh failed for {0}! - status {1}".format(dataset_id, response['Ingestion']['IngestionStatus']))
            break